Dungeon AI V1

In [1]:
# module installation script
%pip install google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# --- IMPORTS ---
import os
import time
import json
import re
import google.generativeai as genai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file!")

genai.configure(api_key=api_key)

# --- GAME STATE ---
model_name = "gemini-2.0-flash"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1
save_file = "savegame.json"

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION ---
def generate_story(context, player_input, difficulty, player_stats, inventory):
    prompt = (
        "You are a fantasy dungeon-master AI. "
        "Continue the adventure in a vivid, immersive style. "
        "Do not repeat the player's action. Keep it concise (max 5 sentences). "
        "After the story, provide any game state updates (health, gold, inventory) in this JSON format:\n"
        "`<META>{\"health\": -10, \"gold\": +5, \"inventory_add\": [\"amulet\"], \"inventory_remove\": [\"torch\"]}</META>`\n"
        "If no update is needed, just write `<META>{}</META>`.\n\n"
        f"Difficulty: {difficulty}\n"
        f"Stats: {player_stats}\n"
        f"Inventory: {inventory}\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )

    try:
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"❌ Error generating story: {e}"

# --- META UPDATE PARSING ---
def apply_meta_updates(text):
    global player_stats, inventory

    meta_match = re.search(r"<META>(.*?)</META>", text, re.DOTALL)
    if not meta_match:
        return text

    story_only = re.sub(r"<META>.*?</META>", "", text, flags=re.DOTALL).strip()

    try:
        updates = json.loads(meta_match.group(1))
        if "health" in updates:
            player_stats["health"] = max(0, player_stats.get("health", 100) + updates["health"])
        if "gold" in updates:
            player_stats["gold"] = max(0, player_stats.get("gold", 0) + updates["gold"])
        if "inventory_add" in updates:
            for item in updates["inventory_add"]:
                if item not in inventory:
                    inventory.append(item)
        if "inventory_remove" in updates:
            for item in updates["inventory_remove"]:
                if item in inventory:
                    inventory.remove(item)
    except Exception as e:
        story_only += f"\n\n❌ Error parsing meta update: {e}"

    return story_only

# --- GAME DISPLAY ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- GAME TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)
    recent_context = "\n".join(game_memory[-6:])
    raw_output = generate_story(recent_context, player_input, difficulty, player_stats, inventory)
    cleaned_output = apply_meta_updates(raw_output)
    context_update = f"\n\n{cleaned_output}"
    context += context_update
    game_memory.append(cleaned_output)
    save_game()
    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()
        display(input_box, submit_button)

# --- SAVE / LOAD / DELETE ---
def save_game():
    data = {
        "context": context,
        "game_memory": game_memory,
        "player_stats": player_stats,
        "inventory": inventory,
        "difficulty": difficulty
    }
    with open(save_file, "w") as f:
        json.dump(data, f)

def load_game():
    global context, game_memory, player_stats, inventory, difficulty
    if not os.path.exists(save_file):
        with output_area:
            clear_output()
            display(Markdown("❌ No save file found!"))
        return
    with open(save_file, "r") as f:
        data = json.load(f)
    context = data["context"]
    game_memory = data["game_memory"]
    player_stats = data["player_stats"]
    inventory = data["inventory"]
    difficulty = data["difficulty"]
    with output_area:
        clear_output()
        display(Markdown("✅ **Game loaded successfully!**"))
        print_game_state()
        display(input_box, submit_button)

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
    with output_area:
        clear_output()
        display(Markdown("🗑️ Save file deleted."))

# --- START NEW GAME ---
def start_new_game(difficulty_choice):
    global context, player_stats, inventory, difficulty, game_memory
    difficulty = {"Easy": 1, "Medium": 2, "Hard": 3}[difficulty_choice]
    if difficulty_choice == "Easy":
        player_stats = {"health": 200, "strength": 15, "gold": 5}
        inventory = ["torch", "wooden sword"]
    elif difficulty_choice == "Medium":
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch", "wooden stick"]
    elif difficulty_choice == "Hard":
        player_stats = {"health": 50, "strength": 5, "gold": 5}
        inventory = ["torch"]
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    save_game()
    with output_area:
        clear_output()
        display(Markdown(f"**New game started on _{difficulty_choice}_ difficulty.**"))
        print_game_state()
        display(input_box, submit_button)

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')
submit_button.on_click(lambda b: (play_turn(input_box.value), setattr(input_box, "value", "")))

output_area = widgets.Output()

# --- GAME MENU ---
difficulty_dropdown = widgets.Dropdown(
    options=['Easy', 'Medium', 'Hard'],
    value='Medium',
    description='Difficulty:',
    layout=widgets.Layout(width='50%')
)
start_button = widgets.Button(description="New Game", button_style='primary')
load_button = widgets.Button(description="Load Game", button_style='info')
delete_button = widgets.Button(description="Delete Save", button_style='danger')

start_button.on_click(lambda b: start_new_game(difficulty_dropdown.value))
load_button.on_click(lambda b: load_game())
delete_button.on_click(lambda b: delete_save())

# --- INITIAL UI DISPLAY ---
if "game_ui_initialized" not in globals():
    display(Markdown("## Welcome to the Fantasy Adventure Game"))
    display(widgets.HBox([difficulty_dropdown, start_button, load_button, delete_button]))
    display(output_area)
    game_ui_initialized = True



## Welcome to the Fantasy Adventure Game

Output()